
O conjunto de dados analisados é sobre pneumonia
https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia?

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # opencv
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score

from keras import preprocessing
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

In [12]:
img = cv2.imread('./fruits-360/Test/Apple/3_100.jpg')

In [13]:
img.shape

(100, 100, 3)

In [14]:
input_dir = './fruits-360/'

In [38]:
#tipo = train ou test
classes = {'Apple':0,'Banana':1}
def ler_dataset(tipo):
  X = []
  y = []
  qtd = 10 if tipo == 'Training' else 5
  #para cada pasta
  for pasta in os.listdir(input_dir+tipo+'/'):
    print(pasta)
    #para cada imagem na pasta
    for img_file in os.listdir(input_dir+tipo+'/'+pasta+'/')[:qtd]:
      #leia a imagem
      img = cv2.imread(input_dir+tipo+'/'+pasta+'/'+img_file)
      #transforme imagem em array de pixels e salve na lista de imagens
      X.append(img)
      #guarde a classificação da imagem
      y.append(classes[pasta])
  X = np.asarray(X)
  y = np.asarray(y)
  return X,y

In [39]:
X_treino,y_treino = ler_dataset('Training')

Banana
Apple


In [40]:
X_treino.shape

(20, 100, 100, 3)

In [41]:
y_treino.shape

(20,)

In [42]:
X_teste,y_teste = ler_dataset('Test')

Banana
Apple


In [43]:
X_teste.shape

(10, 100, 100, 3)

In [44]:
y_teste.shape

(10,)

In [73]:
no_of_classes = len(np.unique(y_treino))
no_of_classes

2

In [74]:
#transformando labels de numérico para categórico
y_treino = to_categorical(y_treino, num_classes = 2)
y_teste = to_categorical(y_teste, num_classes = 2)

In [89]:
X_treino = X_treino.astype('float32')/255
X_teste = X_teste.astype('float32')/255

In [90]:

#definição do modelo
model = Sequential()

# 2 camadas de convolução de 32 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = ( 100, 100, 3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 64 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

# 2 camadas de convolução de 128 filtros + Max pooling [2,2]
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

#  2 camadas fully conected 
model.add(Flatten())#converte os mapas de ativação finais para um veter de 1 dimensão
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "sigmoid"))

In [85]:
# Definindo o algoritmo otmizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [86]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [68]:
# Descrição do modelo
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 439, 712, 32)      2432      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 439, 712, 32)      25632     
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 439, 712, 32)      25632     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 219, 356, 32)      0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 219, 356, 32)      0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 219, 356, 64)      18496     
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 219, 356, 64)     

In [87]:
# treinamento do modelo
model.fit(X_treino, y_treino, epochs=1,verbose = 1)

ValueError: in user code:

    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/traini

In [88]:
model.evaluate(X_teste,y_teste)

ValueError: in user code:

    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/aliny/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/train